# Trading futures by considering the Calmar ratio for performance analysis

The Calmar ratio is a performance metric used to evaluate the risk-adjusted return of an investment strategy.

It was developed by Terry W. Young in 1991 and named after California Managed Accounts Research Inc. (Calmar), where Young was a principal.

The ratio is calculated by dividing the annualized return of an investment strategy by its maximum drawdown.

Professionals in the finance industry use the Calmar ratio to compare the performance of different investment strategies and to assess the risk-adjusted return of a single strategy.

By understanding the Calmar ratio, investors can make more informed decisions about which investment strategies to pursue and how to allocate their capital.

In [1]:
# Imports and set up

# First, import the libraries you’ll use. You can get continuous futures data from the OpenBB SDK. We’ll use Numpy for the math.

from openbb_terminal.sdk import openbb

import numpy as np
# From there, set up a few mock portfolios. For this example, I’ll build an under leveraged futures portfolio and a leveraged futures portfolio. This will demonstrate the effect leverage has on risk.

data = openbb.futures.historical(
    ["ES", "YM", "NQ"],
    start_date="2020-01-01",
    end_date="2022-12-31"
)

futs = data['Adj Close'].pct_change().dropna()

Forecasting Toolkit is disabled. To use the Forecasting features please install the toolkit following the instructions here: https://docs.openbb.co/sdk/quickstart/installation/

Portfolio Optimization Toolkit is disabled. To use the Optimization features please install the toolkit following the instructions here: https://docs.openbb.co/sdk/quickstart/installation/

In [2]:
# This code get continuous futures data for the E-Mini S&P 500 Index, E-Mini Dow Jones Industrial Average, and E-Mini Nasdaq 100. The settlement price is returned in the “Adj Close” column. Use it to compute the daily returns.

# Let’s create two mock portfolios.

port_1 = futs.ES * 0.60 + futs.YM * 0.10 + futs.NQ * 0.10

port_2 = futs.ES * 0.90 + futs.YM * 0.15 + futs.NQ * 0.15
# Apply a weighting to each of the symbols. Note the weights add up to less that 1 for port_1 and greater than 1 for port_2. This is an easy way to replicate an under- and over-leveraged portfolio. Also note that the weights for port_2 are 1.5x that of port_1.

In [3]:
# Compute the Calmar ratio

# Next, write a function that computes the Calmar ratio.

def ann_return(returns):
    ending_value = (returns + 1).prod()
    num_years = len(returns) / 252  # daily returns
    ann_return = ending_value ** (1/num_years) - 1
    return ann_return


def calmar_ratio(returns):

    # cumulative returns
    cumulative_returns = (returns + 1).cumprod() * 100

    # max drawdown
    max_return = np.fmax.accumulate(cumulative_returns)
    max_dd = ((cumulative_returns - max_return) / max_return).min()

    # annualized return
    ann_ret = ann_return(returns)

    return ann_ret / abs(max_dd)

In [5]:
# Compare portfolio performance

# Check out the results.

# port1
ret = ann_return(port_1)
p1 = calmar_ratio(port_1)

# port2
ret = ann_return(port_2)
p2 = calmar_ratio(port_2)

print(f"Portfolio 1: {p1:.2f}")
print(f"Portfolio 2: {p2:.2f}")


Portfolio 1: 0.19
Portfolio 2: 0.16
